In [5]:
# Dependencies
import pandas as pd
import hvplot.pandas
import geopandas as gpd
from datetime import datetime

In [7]:
# Create GeoDataFrame from local geojson file downloaded from CalFire
yr = 2023
gdf = gpd.read_file(f'../outputs/calfire-geojson/ca_fire_{yr}.geojson')

In [3]:
gdf['CAUSE_ID'] = gdf['CAUSE'].astype(int)
gdf['CAUSE'] = gdf['CAUSE'].astype(str)

In [4]:
lst = ['Lightning','Equipment Use','Smoking','Campfire','Debris','Railroad','Arson','Playing with fire','Miscellaneous',
       'Vehicle','Powerline','Firefighter Training','Non-Firefighter Training','Unknown/Unidentified','Structure',
       'Aircraft','','Escaped Prescribed Burn','Illegal Alien Campfire']

In [9]:
fire_gdf = gdf[['ALARM_DATE','CONT_DATE','CAUSE_ID','CAUSE','GIS_ACRES','AGENCY','FIRE_NAME','geometry']]
fire_gdf = fire_gdf.dropna().reset_index(drop=True)

In [ ]:
for i in range(0,fire_gdf['CAUSE'].count()):
    fire_gdf.loc[i,'ALARM_DATE'] = str(datetime.strptime(str(fire_gdf.loc[i,'ALARM_DATE']),'%a, %d %b %Y %H:%M:%S %Z').date())
    fire_gdf.loc[i,'CONT_DATE'] = str(datetime.strptime(str(fire_gdf['CONT_DATE'][i]),'%a, %d %b %Y %H:%M:%S %Z').date())
    fire_gdf.loc[i,'CAUSE'] = lst[int(fire_gdf.loc[i,'CAUSE_ID']-1)]

In [11]:
fire_gdf.sort_values(['ALARM_DATE','CONT_DATE','GIS_ACRES']).reset_index(drop=True)

,ALARM_DATE,CONT_DATE,CAUSE_ID,CAUSE,GIS_ACRES,AGENCY,FIRE_NAME,geometry
0,2023-03-11,2023-03-12,14,Unknown/Unidentified,15.253180,BLM,Winterhaven,"POLYGON ((-114.58925 32.75259, -114.58860 32.7..."
1,2023-04-06,2023-04-06,11,Powerline,54.547180,CDF,MAIN,"POLYGON ((-118.39118 37.37759, -118.39187 37.3..."
2,2023-04-17,2023-04-22,10,Vehicle,62.934320,BLM,IMVITE,"MULTIPOLYGON (((-116.47591 36.43560, -116.4759..."
3,2023-04-20,2023-04-22,14,Unknown/Unidentified,19.897410,BLM,Laguna,"POLYGON ((-114.50716 32.82059, -114.50835 32.8..."
4,2023-04-26,2023-05-11,14,Unknown/Unidentified,226.975100,USF,NOB,"POLYGON ((-117.56658 34.29260, -117.56634 34.2..."
...,...,...,...,...,...,...,...,...
278,2023-12-15,2023-12-16,14,Unknown/Unidentified,169.973000,CCO,PINE,"MULTIPOLYGON (((-118.86844 35.55474, -118.8684..."
279,2023-12-16,2023-12-16,14,Unknown/Unidentified,1.008626,CDF,FRANKLIN,"POLYGON ((-120.63869 35.58811, -120.63872 35.5..."
280,2023-12-16,2023-12-16,10,Vehicle,3.066564,CDF,PANZA,"MULTIPOLYGON (((-120.13459 35.38116, -120.1337..."
281,2023-12-16,2023-12-16,10,Vehicle,13.342450,CDF,58 CARRIZO,"MULTIPOLYGON (((-120.05328 35.36509, -120.0531..."


In [8]:
fire_gdf.to_file(f'../outputs/calfire-geojson/ca_fire_{yr}.geojson',driver="GeoJSON")

In [13]:
fire_map = fire_gdf.hvplot(
    frame_width = 700, frame_height = 700,
    geo = True, coastline = True,
    tiles = "EsriImagery", tiles_opts = {'alpha': 0.1},
    color='GIS_ACRES', cmap='plasma', alpha=0.6,
    hover_cols=['FIRE_NAME','CAUSE']
)
fire_map

:Overlay
   .WMTS.I      :WMTS   [Longitude,Latitude]
   .Polygons.I  :Polygons   [Longitude,Latitude]   (GIS_ACRES,FIRE_NAME,CAUSE)
   .Coastline.I :Feature   [Longitude,Latitude]